In [1]:
import rsudp.raspberryshake as rs
from obspy.core.stream import Stream
import obspy as obs
from obspy import read

import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
from IPython.display import display, clear_output


In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sjaak/m

In [2]:
rs.initRSlib() # running this twice in a row gives an error. Restart kernel in that case
s = Stream()
plt.ion()
fig = plt.figure()
ax1 = fig.add_subplot(3, 1, 1) # Z-component 
ax2 = fig.add_subplot(3, 1, 2) # E-component
ax3 = fig.add_subplot(3, 1, 3) # N-component

def anipause():
    global pause
    pause = True

def aniresume():
    global pause
    pause = False

[get_inventory] WARNING: No station name given, continuing without inventory.


In [3]:
# plt.rcParams['fig.labelsize'] = 'x-large'
# plt.rcParams('axes', labelsize='20')    # fontsize of the x and y labels

global pause
pause = False 

# This function is called periodically from FuncAnimation
def animate(i):
    """
    Function to be called by animation.FuncAnimation(). This function provides 
    for each frame to be plotted the relevant data. Plots Z, N and E-component data 
    retrieved from the connected Raspberry Shake seismometer. 
    
    """
    for i in range(3):
        d = rs.getDATA() # regardless of 'pause', we keep updating the data
        if pause == False:        
            rs.update_stream(s, d) # only for pause==False, we actually write 
                                   # the data to the plotting variable
    trE = s.select(channel="EHE")[0] # index [0] to obtain only trace present inside stream
    trN = s.select(channel="EHN")[0]
    trZ = s.select(channel="EHZ")[0]
    
    fig.canvas.draw()
    fig.canvas.flush_events()
    
    et = trZ.stats.endtime
    delta=12 # plotting time window in seconds 
    newet = et-delta
    trZ = trZ.slice(starttime=newet, endtime=et) # trim time-window of data
    trE = trE.slice(starttime=newet, endtime=et)
    trN = trN.slice(starttime=newet, endtime=et)
    
    ax1.clear() # plot Z-trace
    ax1.plot(trZ.times("matplotlib"), trZ.data, "k-", lw=0.5)
    ax1.xaxis_date()
    ax1.legend(['Vertical component [AM.Z0000.00.EHZ]'], loc='upper left')
    ax1.set_xlabel('Time')
    # ax1.set_xticks(rotation=45, ha='right')
    
    ax2.clear() # plot E-trace
    ax2.plot(trE.times("matplotlib"), trE.data, "k-", lw=0.5)
    ax2.xaxis_date()
    ax2.legend(['East-West component [AM.Z0000.00.EHE]'], loc='upper left')
    ax2.set_xlabel('Time')#, fontsize=)
    ax2.set_ylabel('Amplitude')
    # ax2.set_xticks(rotation=45, ha='right')
    
    ax3.clear() # plot N-trace
    ax3.plot(trN.times("matplotlib"), trN.data, "k-", lw=0.5)
    ax3.xaxis_date()
    ax3.legend(['North-South component [AM.Z0000.00.EHN]'], loc='upper left')
    ax3.set_xlabel('Time')
    # ax3.set_xticks(rotation=45, ha='right')
    
    ax1.sharey(ax2)
    ax3.sharey(ax2)
    fig.autofmt_xdate()  
    plt.xticks(rotation=45, ha='right')

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(fig, animate, interval=100)
plt.show()

In [5]:
anipause()

In [6]:
aniresume()